In [ ]:
import nltk
import re

def input_output_file(filename,tokenname, token):
    string_output = str(str(tokenname) + str(filename) +".txt")
    outputfile = open(string_output, mode="w", encoding="utf-8")
    string_input = str(str(filename)+".txt")
    myfile = open(string_input, mode="r", encoding="utf-8")
    outputfile.write(str(filename) + "\n")
    text = myfile.read()
    myfile.close()
    text_tokenize = nltk.sent_tokenize(text)
    fetch_sentence(text_tokenize, token, outputfile)
modal = ["pé", "pe", "dwèt", "dwet", "dwé","dwe","anvi","sa","bizwen", "lé", "vlé"]
expletive = ["fok", "foc", "fo", "se", "sé"]
negation = ["ayen", "pèsonn", "janme", "jamais", "personne", "rien"]
space = ["\n", "\r"]
modal_exp = ["pé"]

def fetch_sentence(text, token, outputfile):
    cnt = 0
    token_sentences = []
    for sentence in text:
        sentence_word = nltk.word_tokenize(sentence)
        cnt += 1
        for x in sentence_word :
            if x in token:
                sentence_indexed = str(cnt)+" "+sentence
                if sentence_indexed not in token_sentences:
                    token_sentences.append(sentence_indexed)
                    outputfile.write(sentence_indexed)
                    outputfile.write("\n")
                else:
                    continue
    for x in token_sentences:
        print(x)
    outputfile.close()

#input_output_file("aujourdhuidimanche", "negation", negation)
#input_output_file("aujourdhuidimanche", "expletive", expletive)
#input_output_file("aujourdhuidimanche", "negation", negation)
#input_output_file("aujourdhuidimanche", "expletive", expletive)
input_output_file("haiticorpus", "modal_exp", modal_exp)
input_output_file("NEWANTILLA_2009", "modal_exp", modal_exp)
input_output_file("NEWAkreyolad_2004", "modal_exp", modal_exp)

#for x in negative_sentences:
 #   print (x)
#print(negative_sentences)

In [5]:
import os
from pathlib import Path
import nltk
import csv


def write_file(token, sentences_dict):
    OUT = open(token+"_output.csv", mode="w", encoding="utf-8")
    for sentence in sentences_dict:
        OUT.write(sentences)
        
        
def access_data(out_file, datasets,targets_mart, targets_hait):
    for dataset in datasets:
        sources = Path(dataset).glob('*')
        for source in sources:
            time_stamps = Path(source).glob('*')
            for time_stamp in time_stamps:
                files = Path(time_stamp).glob('*.txt')
                for file in files:
                    #print(file)
                    file_descr = str(file).split("\\")
                    filename = str(file_descr[-1])
                    time_stamp_label = str(file_descr[-2])
                    source_label = str(file_descr[-3])
                    dataset_label = str(file_descr[-4])
                    #print(dataset_label, source_label, time_stamp_label, filename)
                    extract_data(out_file, dataset_label, source_label, time_stamp_label, file, filename, targets_mart, targets_hait)
    
                    
def extract_data(out_file, dataset, source, time_stamp, file, filename,targets_mart, targets_hait):
    sentences = []
    datawriter = csv.writer(out_file, delimiter='\t')
    with open(file, mode="r", encoding="utf-8", errors ="replace") as f:
        text = f.read()
        f.close()
        text = nltk.sent_tokenize(text)
        cnt = 0
        if dataset == "martinican_creole" :
            for sentence in text:
                text_word = nltk.word_tokenize(sentence)
                for target in targets_mart:
                    for word in range(len(text_word)-1):
                        if target == str(text_word[word]+" "+text_word[word+1]):
                            cnt +=1
                            datawriter.writerow([dataset, source, time_stamp, filename, " ".join(text_word[0:word]), text_word[word]+" "+text_word[word+1], " ".join(text_word[word+2:])])
                        elif target == text_word[word]: 
                            cnt +=1
                            datawriter.writerow([dataset, source, time_stamp, filename, " ".join(text_word[0:word]), text_word[word], " ".join(text_word[word+1:])])           
        elif dataset == "haitian_creole" :
            for sentence in text:
                text_word = nltk.word_tokenize(sentence)
                for target in targets_hait:
                    for word in range(len(text_word)-1):
                        if target == str(text_word[word]+" "+text_word[word+1]):
                            cnt +=1
                            datawriter.writerow([dataset, source, time_stamp, filename, " ".join(text_word[0:word]), text_word[word]+" "+text_word[word+1], " ".join(text_word[word+2:])])
                        elif target == text_word[word]: 
                            cnt +=1
                            datawriter.writerow([dataset, source, time_stamp, filename, " ".join(text_word[0:word]), text_word[word], " ".join(text_word[word+1:])]) 
    del datawriter
                            #article_url_dict[author].append(article_url)
            #print(sentences)
                    

def main():
    datasets = ["martinican_creole", "haitian_creole"]
    targets_hait = ["ta"]
    targets_mart = ["ké"]
    #targets_mart = ["pé pa", "pa pé"]
    #targets_hait = []
    out_file = open("dataset_irrealis.txt", mode="w", encoding="utf-8", newline='', errors="replace")
    access_data(out_file, datasets,targets_mart, targets_hait)
    out_file.close()
main() 

In [30]:
import os
from pathlib import Path
import nltk
import csv
from collections import Counter
from nltk.tokenize import RegexpTokenizer

def write_file(author, sentences_dict):
    OUT = open(author +"_stats.csv", mode="w", encoding="utf-8")
    for sentence in sentences_dict:
        OUT.write(sentences)
        
        
def access_data(datasets):
    for dataset in datasets:
        authors = Path(dataset).glob('*')
        if dataset == "martinican_creole":
            print("ok dataset")
            for author in authors:
                author_corpus = Path(author).glob('*')
                for folder in author_corpus:
                    frequency_distribution = 0
                    count_type = 0
                    count_token = 0
                    if "data_Creole" in str(folder):
                        print("ok author_corpus")
                        files = Path(folder).glob('*.txt')
                        for file in files:
                            if frequency_distribution == 0:
                                frequency_distribution = extract_data_stats(file)
                            if frequency_distribution != 0:
                                frequency_distribution.update(extract_data_stats(file)) 
                        for token, frequency in sorted(frequency_distribution.items()):
                            count_type +=1
                            count_token = count_token + int(frequency)
                        print(dataset, author, folder, count_type, count_token)
                        count_type = 0
                        count_token = 0   
        if dataset == "haitian_creole":
            print("ok dataset")
            for author in authors:
                frequency_distribution = 0
                count_type = 0
                count_token = 0
                date_corpus = Path(author).glob('*')
                for folder in date_corpus:
                    files = Path(folder).glob('*.txt')
                    for file in files:
                        if frequency_distribution == 0:
                            frequency_distribution = extract_data_stats(file)
                        if frequency_distribution != 0:
                            frequency_distribution.update(extract_data_stats(file)) 
                for token, frequency in sorted(frequency_distribution.items()):
                    count_type +=1
                    count_token = count_token + int(frequency)
                print(dataset, author, folder, count_type, count_token)                       
def extract_data_stats(file):
    tokenizer = RegexpTokenizer(r'[a-zA-Z]+')
    with open(file, mode="r", encoding="utf-8", errors ="replace") as f:
        text = f.read()
        f.close()
        text = text.lower()
        tokens = tokenizer.tokenize(text)
        for token in tokens:
            if token == r'[0-9]+':
                del token
        #tokens = nltk.word_tokenize(text)
        frequency_distribution = nltk.FreqDist(tokens)
        #token_frequency_profile = dict(frequency_distribution)
    return frequency_distribution

def main():
    datasets = ["haitian_creole"]
    access_data(datasets)
    
main() 

ok dataset
haitian_creole haitian_creole\alterpresse_dataset haitian_creole\alterpresse_dataset\9_septembre_2022 18042 655365
haitian_creole haitian_creole\haitian_creole_wiki_corpus haitian_creole\haitian_creole_wiki_corpus\2011-2021 19352 411448
haitian_creole haitian_creole\haitiliberte_dataset haitian_creole\haitiliberte_dataset\September_9,_2022 9306 164717
haitian_creole haitian_creole\rezonodwes_dataset haitian_creole\rezonodwes_dataset\9_septembre_2022 12821 187984


In [1]:
import os
from pathlib import Path
import nltk
import csv


def write_file(token, sentences_dict):
    OUT = open(token+"_output.csv", mode="w", encoding="utf-8")
    for sentence in sentences_dict:
        OUT.write(sentences)
        
        
def access_data(out_file, datasets,targets_mart, targets_hait):
    for dataset in datasets:
        sources = Path(dataset).glob('*')
        for source in sources:
            #print(source)
            time_stamps = Path(source).glob('*')
            for time_stamp in time_stamps:
                #print(time_stamp)
                time_stamp_path = str(time_stamp).split("\\")
                #print(time_stamp_path[-1])
                if time_stamp_path[-1] == "data_Creole" or time_stamp_path[-1] == "data_French":
                    continue
                else: 
                    #print(time_stamp)
                    files = Path(time_stamp).glob('*.txt')
                    for file in files:
                        print(file)
                        file_descr = str(file).split("\\")
                        filename = str(file_descr[-1])
                        time_stamp_label = str(file_descr[-2])
                        source_label = str(file_descr[-3])
                        dataset_label = str(file_descr[-4])
                        #print(dataset_label, source_label, time_stamp_label, filename)
                        extract_data(out_file, dataset_label, source_label, time_stamp_label, file, filename, targets_mart, targets_hait)
    
                    
def extract_data(out_file, dataset, source, time_stamp, file, filename,targets_mart, targets_hait):
    sentences = []
    datawriter = csv.writer(out_file, delimiter='\t')
    with open(file, mode="r", encoding="utf-8", errors ="replace") as f:
        text = f.read()
        f.close()
        text = nltk.sent_tokenize(text)
        for sentence in text:
            text_word = nltk.word_tokenize(sentence)
            datawriter.writerow([dataset, source, time_stamp, filename, " ".join(text_word)])
        del datawriter
                            #article_url_dict[author].append(article_url)
            #print(sentences)
                    

def main():
    datasets = ["martinican_creole"]
    targets_hait = ["fòk", "Fòk"]
    targets_mart = ["fé fret", "fé cho", "ni soley", "ni lapli", "Fodré", "Fodra"]
    #targets_mart = ["pé pa", "pa pé"]
    #targets_hait = []
    out_file = open("dataset_mar_full.txt", mode="w", encoding="utf-8", newline='', errors="replace")
    access_data(out_file, datasets,targets_mart, targets_hait)
    out_file.close()
main() 

martinican_creole\Antilla\2009-2018\ANTILLA.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman0.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman1.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman10.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman11.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman12.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman13.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman14.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman15.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman16.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman17.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman18.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman19.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman2.txt
martinican_creole\Daniel_Boukman\2007to2021\Daniel_Boukman20.txt
martinican_creole\Daniel_Boukman\2007to2021\D

martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery25.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery26.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery27.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery28.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery29.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery3.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery30.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery31.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery32.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery33.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery34.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery35.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery4.txt
martinican_creole\Hughes_Barthelery\2016to2021\Hughes_Barthelery5.txt
martinica

martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide31.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide32.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide33.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide34.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide35.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide36.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide37.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide38.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide39.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide4.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide40.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide41.txt
martinican_creole\Jean_Laurent_Alcide\2007-2022\Jean_Laurent_Alcide42.txt
martinican_creole\Jean_Laurent_Alcide\2

martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha54.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha55.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha56.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha57.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha58.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha59.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha6.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha60.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha61.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha62.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha63.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha64.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha65.txt
martinican_creole\Leandre_Litampha\2016-2021\Leandre_Litampha66.txt
martinican_creole\Leandre_Litampha\2016-2021\Lean

martinican_creole\raphael_confiant\2006-2021\raphael_confiant167.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant168.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant169.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant17.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant170.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant171.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant172.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant173.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant174.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant175.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant176.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant177.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant178.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant179.txt
martinican_creole\raphael_confiant\

martinican_creole\raphael_confiant\2006-2021\raphael_confiant277.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant278.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant279.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant28.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant280.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant281.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant282.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant283.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant284.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant285.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant286.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant287.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant288.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant289.txt
martinican_creole\raphael_confiant\

martinican_creole\raphael_confiant\2006-2021\raphael_confiant385.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant386.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant387.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant388.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant389.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant39.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant390.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant391.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant392.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant393.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant394.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant395.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant396.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant397.txt
martinican_creole\raphael_confiant\

martinican_creole\raphael_confiant\2006-2021\raphael_confiant83.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant84.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant85.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant86.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant87.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant88.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant89.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant9.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant90.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant91.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant92.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant93.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant94.txt
martinican_creole\raphael_confiant\2006-2021\raphael_confiant95.txt
martinican_creole\raphael_confiant\2006-2021\raph

martinican_creole\therese_leotin\2015-2020\therese_leotin26.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin27.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin28.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin29.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin3.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin30.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin31.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin32.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin33.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin34.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin35.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin36.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin37.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin38.txt
martinican_creole\therese_leotin\2015-2020\therese_leotin39.txt
martinican_creole\therese_leotin\2015-202